In [3]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
# for reading images
import PIL.Image

/Users/ryanrodrigues/.local/share/virtualenvs/GeminiGenerativeAI-r0q8n7Y_/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

### Image to Text

In [5]:

organ = PIL.Image.open("./image/cookies.jpeg")
response = model.generate_content(["Tell me about this instrument", organ])
print(response.text)

Those are coconut cookies.  Specifically, they appear to be a type of coconut macaroon or a similar cookie where shredded coconut is a prominent ingredient, baked into a relatively thin, flat, and somewhat crisp cookie.  The slight browning suggests they might have a touch of brown sugar or a similar ingredient contributing to their flavor.



### Chat 

In [6]:
# history initializes the chat with a predefined conversation history
# The history contains two messages:
#   The first message is from the user: "Hello".
#   The second message is from the model: "Great to meet you. What would you like to know?".
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)

# Chat remebers the previous conversation and provides context to messages sent.
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response2 = chat.send_message("How many paws are in my house?")
print(response2.text)

That's lovely!  Do you have any questions about your dogs, or would you like to tell me more about them?  Perhaps their breeds, names, or favorite things?

There are eight paws in your house.  Two dogs x four paws/dog = eight paws.



### Prompting a Video

**Download a Video**

In [11]:
# !wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4 -O GreatRedSpot.mp4
import wget

url = "https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4"
file = wget.download(url)

print(f"Downloaded file : {file}")

Downloaded file : GreatRedSpot.mp4


**Upload a file to Gemini Generative Ai services storage**
<br/>The location where they are uploaded are abstracted from the user.

In [ ]:
# Upload the video and print a confirmation.
print(f"Uploading file...")
video_file = genai.upload_file(path=file)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/2zuwr1y7ynba


**Check if Video has been uploaded**

In [13]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

.

**Prompt with a video and text**

In [15]:
from IPython.display import Markdown

# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

Making LLM inference request...


This video explores Jupiter and its Great Red Spot. Jupiter is the largest and oldest planet in the solar system, and it’s made of the same elements as a star, though it never grew massive enough to ignite. Its swirling interior of gases and liquids creates the distinctive colorful cloud bands across the planet. The Great Red Spot, an anticyclone, has raged on Jupiter for over a century. As Jupiter is composed of gas and has no land mass, the storm continues with nothing to slow it down. 

Scientists have observed the Great Red Spot shrinking, darkening, and getting rounder. Initially, they thought this meant the wind speeds within the spot were increasing. However, data now shows that it’s not spinning faster but growing taller. The smaller circumference from above is due to the spot increasing in height. It’s like shaping pottery. As the pottery spins on a wheel, its height increases. From our perspective, the top appears to become smaller and rounder.

When it was first observed, the Great Red Spot was large enough to hold three Earths. Currently, the spot is only a little larger than Earth. These discoveries come from analyzing data collected by numerous NASA missions, including Voyager, Hubble, and Juno. Scientists hope further research will unlock more secrets of the Great Red Spot.


Here’s a quiz about the video:

1. True or False: Jupiter is the oldest planet in the solar system.
2. How old is Jupiter?
3. True or False: Jupiter is made of the same elements as a planet.
4. What creates Jupiter’s distinctive bands of color?
5. What type of storm is the Great Red Spot?
6. What has caused the Great Red Spot to continue for over a century?
7. True or False: As the Great Red Spot shrinks, it’s spinning faster.
8. True or False: The Great Red Spot is getting taller.
9. What analogy does the video use to explain what’s happening to the Great Red Spot?
10. Name two of the NASA missions that have collected data used to study the Great Red Spot.

Answer Key:
1. True
2. 4.5 billion years
3. False
4. Its swirling interior of gases and liquids.
5. Anticyclone
6. The absence of land masses.
7. False
8. True
9. Shaping pottery
10. Voyager, Hubble, and Juno.